In [1]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [2]:
pd.set_option('display.max_columns', None)

### setup

Reading in the data file provided by the Department of Public Works on July 10, 2025. This dataset includes all unaccepted streets.

In [3]:
df_dpw = pd.read_csv('./data/dpw_data.csv')

In [4]:
df_dpw.columns = df_dpw.columns.str.lower().str.replace(' ', '_')

In [5]:
df_dpw

,cnn,street_name,location,x_coord,y_coord,latitude,longitude,bos_district,zip_code,sfar_neighborhood_desc,jurisdiction,layer,bike_path,accepted
0,110000,01ST ST,01ST ST: CLEMENTINA ST to FOLSOM ST (245 - 299),6014174.520,2114753.853,37.787508,-122.394745,6,94105,South Beach,CALTRANS/DPW,STREETS,No,N
1,114000,01ST ST,01ST ST: HARRISON ST \ I-80 E ON RAMP to END (...,6014784.513,2114116.436,37.785792,-122.392589,6,94105,South Beach,CALTRANS/DPW,STREETS,No,N
2,140000,02ND ST,02ND ST: HARRISON ST to STILLMAN ST (400 - 459),6014173.689,2113442.524,37.783907,-122.394656,6,94107,South of Market,CALTRANS/DPW,STREETS,Yes,N
3,150000,03RD AVE,03RD AVE: END to LAKE ST (1 - 99),5994888.446,2114976.446,37.787028,-122.461486,1,94118,Lake,DPW,STREETS,No,N
4,159000,03RD AVE,03RD AVE: KEZAR DR \ LINCOLN WAY to LINCOLN WA...,5995166.956,2107405.925,37.766257,-122.459974,7,94117,Inner Sunset,DPW,STREETS,Yes,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,15329000,SUNNYDALE AVE,SUNNYDALE AVE: SANTOS ST to END (0 - 0),6006743.014,2087633.962,37.712627,-122.418532,,,,DPW,PAPER,No,N
1946,15331000,SUNRISE WAY,SUNRISE WAY: MALOSI ST to SANTOS ST (0 - 0),6006690.422,2086843.851,37.710454,-122.418658,,,,DPW,PAPER,No,N
1947,14348000,MACKY LN,MACKY LN: GOLDEN BELL WAY to END (0 - 0),6021870.888,2127869.609,37.823946,-122.369023,,,,DPW,STREETS,No,N
1948,14291000,MAYBECK ST,MAYBECK ST: AVENUE OF THE PALMS to GARDEN WALK...,6019867.686,2128150.234,37.824606,-122.375976,,,,DPW,STREETS,No,N


reading in a dataset that contains **all** streets in San Francisco. This data was downloaded from [S.F. Open Data](https://data.sfgov.org/Geographic-Locations-and-Boundaries/Streets-Active-and-Retired/3psu-pn9h/about_data) on July 10, 2025. 

In [6]:
df_streets = pd.read_csv('./data/Streets___Active_and_Retired_20250709.csv')

In [7]:
df_streets.head()

,cnn,lf_fadd,lf_toadd,rt_fadd,rt_toadd,street,st_type,f_st,t_st,f_node_cnn,t_node_cnn,accepted,active,classcode,date_added,date_altered,date_dropped,gds_chg_id_add,gds_chg_id_altered,gds_chg_id_dropped,jurisdiction,layer,nhood,oneway,street_gc,streetname,streetname_gc,zip_code,analysis_neighborhood,supervisor_district,line,data_as_of,data_loaded_at
0,9999000,301.0,399.0,300.0,398.0,ORTEGA,ST,10TH AVE,11TH AVE,27098000.0,27113000.0,True,True,5.0,07/01/1998,NaN,NaN,NaN,NaN,NaN,DPW,STREETS,Golden Gate Heights,B,ORTEGA,ORTEGA ST,ORTEGA ST,94122.0,Inner Sunset,7.0,"LINESTRING (-122.466600499 37.752803078, -122....",2025/07/09 03:46:00 AM,2025/07/09 10:11:28 AM
1,10000202,501.0,599.0,0.0,0.0,ORTEGA,ST,AERIAL WAY,CASCADE WALK,32870000.0,32871000.0,True,True,5.0,07/01/1998,08/14/2013,NaN,NaN,2013-028,NaN,DPW,STREETS,Golden Gate Heights,T,ORTEGA,ORTEGA ST,ORTEGA ST,94122.0,Inner Sunset,7.0,"LINESTRING (-122.468335072 37.753243731, -122....",2025/07/09 03:46:00 AM,2025/07/09 10:11:28 AM
2,10002000,801.0,899.0,800.0,898.0,ORTEGA,ST,15TH AVE,16TH AVE,27308000.0,27310000.0,True,True,5.0,07/01/1998,NaN,NaN,NaN,NaN,NaN,DPW,STREETS,Inner Sunset,B,ORTEGA,ORTEGA ST,ORTEGA ST,94122.0,Inner Sunset,7.0,"LINESTRING (-122.472007575 37.752569818, -122....",2025/07/09 03:46:00 AM,2025/07/09 10:11:28 AM
3,10010000,1501.0,1599.0,1500.0,1598.0,ORTEGA,ST,22ND AVE,23RD AVE,27338000.0,27379000.0,True,True,5.0,07/01/1998,NaN,NaN,NaN,NaN,NaN,DPW,STREETS,Central Sunset,B,ORTEGA,ORTEGA ST,ORTEGA ST,94122.0,Sunset/Parkside,4.0,"LINESTRING (-122.479557564 37.752234616, -122....",2025/07/09 03:46:00 AM,2025/07/09 10:11:28 AM
4,10019000,2401.0,2499.0,2400.0,2498.0,ORTEGA,ST,31ST AVE,32ND AVE,27674000.0,27677000.0,True,True,5.0,07/01/1998,NaN,NaN,NaN,NaN,NaN,DPW,STREETS,Central Sunset,B,ORTEGA,ORTEGA ST,ORTEGA ST,94122.0,Sunset/Parkside,4.0,"LINESTRING (-122.489193569 37.751806869, -122....",2025/07/09 03:46:00 AM,2025/07/09 10:11:28 AM


Let's keep the following columns:

In [8]:

df_streets = df_streets[['line', 'cnn', 'active', 'supervisor_district']]

In [9]:
df_streets

,line,cnn,active,supervisor_district
0,"LINESTRING (-122.466600499 37.752803078, -122....",9999000,True,7.0
1,"LINESTRING (-122.468335072 37.753243731, -122....",10000202,True,7.0
2,"LINESTRING (-122.472007575 37.752569818, -122....",10002000,True,7.0
3,"LINESTRING (-122.479557564 37.752234616, -122....",10010000,True,4.0
4,"LINESTRING (-122.489193569 37.751806869, -122....",10019000,True,4.0
...,...,...,...,...
17150,"LINESTRING (-122.389022427 37.775404055, -122....",15287000,True,6.0
17151,"LINESTRING (-122.373499523 37.734495279, -122....",6535000,True,NaN
17152,"LINESTRING (-122.423353436 37.709250951, -122....",6150201,True,NaN
17153,"LINESTRING (-122.378784033 37.712929869, -122....",6179000,True,NaN


Since the data from DPW does not include a geometry column with line segments, we're going to use the geometry column from the streets dataset. We'll merge the two datasets on the "CNN" column, which is the unique identifier for each street segment.

In [10]:
df_dpw = df_dpw.merge(df_streets, on='cnn', how='left')

In [11]:
df_dpw

,cnn,street_name,location,x_coord,y_coord,latitude,longitude,bos_district,zip_code,sfar_neighborhood_desc,jurisdiction,layer,bike_path,accepted,line,active,supervisor_district
0,110000,01ST ST,01ST ST: CLEMENTINA ST to FOLSOM ST (245 - 299),6014174.520,2114753.853,37.787508,-122.394745,6,94105,South Beach,CALTRANS/DPW,STREETS,No,N,"LINESTRING (-122.395007842 37.787717615, -122....",True,6.0
1,114000,01ST ST,01ST ST: HARRISON ST \ I-80 E ON RAMP to END (...,6014784.513,2114116.436,37.785792,-122.392589,6,94105,South Beach,CALTRANS/DPW,STREETS,No,N,"LINESTRING (-122.392936685 37.786067532, -122....",True,6.0
2,140000,02ND ST,02ND ST: HARRISON ST to STILLMAN ST (400 - 459),6014173.689,2113442.524,37.783907,-122.394656,6,94107,South of Market,CALTRANS/DPW,STREETS,Yes,N,"LINESTRING (-122.395160622 37.784311013, -122....",True,6.0
3,150000,03RD AVE,03RD AVE: END to LAKE ST (1 - 99),5994888.446,2114976.446,37.787028,-122.461486,1,94118,Lake,DPW,STREETS,No,N,"LINESTRING (-122.461501479 37.787239798, -122....",True,1.0
4,159000,03RD AVE,03RD AVE: KEZAR DR \ LINCOLN WAY to LINCOLN WA...,5995166.956,2107405.925,37.766257,-122.459974,7,94117,Inner Sunset,DPW,STREETS,Yes,N,"LINESTRING (-122.459931488 37.76634884, -122.4...",True,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,15329000,SUNNYDALE AVE,SUNNYDALE AVE: SANTOS ST to END (0 - 0),6006743.014,2087633.962,37.712627,-122.418532,,,,DPW,PAPER,No,N,"LINESTRING (-122.418038222 37.712487948, -122....",True,10.0
1946,15331000,SUNRISE WAY,SUNRISE WAY: MALOSI ST to SANTOS ST (0 - 0),6006690.422,2086843.851,37.710454,-122.418658,,,,DPW,PAPER,No,N,"LINESTRING (-122.418243703 37.710337536, -122....",True,10.0
1947,14348000,MACKY LN,MACKY LN: GOLDEN BELL WAY to END (0 - 0),6021870.888,2127869.609,37.823946,-122.369023,,,,DPW,STREETS,No,N,"LINESTRING (-122.368222763 37.822730801, -122....",True,6.0
1948,14291000,MAYBECK ST,MAYBECK ST: AVENUE OF THE PALMS to GARDEN WALK...,6019867.686,2128150.234,37.824606,-122.375976,,,,DPW,STREETS,No,N,"LINESTRING (-122.376249932 37.824351379, -122....",True,6.0


Out of the 1938 unaccepted streets, the following did not match a street segment in the streets dataset. 

In [12]:
df_dpw[df_dpw['line'].isna()]

,cnn,street_name,location,x_coord,y_coord,latitude,longitude,bos_district,zip_code,sfar_neighborhood_desc,jurisdiction,layer,bike_path,accepted,line,active,supervisor_district
1440,10830002,NaN,NaN,NaN,NaN,NaN,NaN,10,94124,Bayview District,DPW,NaN,No,N,NaN,NaN,NaN
1843,15320000,TREASURE ISLAND RD,TREASURE ISLAND RD: I-80 E OFF RAMP \ I-80 W O...,6022434.789,2122471.247,37.809154,-122.366698,,,,DPW,STREETS,No,N,NaN,NaN,NaN
1848,15336000,CHUMASERO DR,CHUMASERO DR: END to FONT BLVD (0 - 0),5991141.029,2088883.846,37.715168,-122.472548,,,,DPW,PAPER,No,N,NaN,NaN,NaN
1849,15337000,CHUMASERO DR,CHUMASERO DR: FONT BLVD to END (0 - 0),5990846.290,2088551.182,37.714238,-122.473543,,,,DPW,PAPER,No,N,NaN,NaN,NaN
1852,15308000,MACALLA RD,MACALLA RD: END to I-80 W ON RAMP \ NORTH GATE...,6023218.504,2123420.124,37.811802,-122.364051,,,,DPW,STREETS,No,N,NaN,NaN,NaN
1853,15309000,TREASURE ISLAND RD,TREASURE ISLAND RD: END to END (670 - 699),6023361.507,2122512.999,37.809319,-122.363494,,,,DPW,STREETS,No,N,NaN,NaN,NaN
1854,15310000,TREASURE ISLAND RD,TREASURE ISLAND RD: END to END (700 - 750),6023385.570,2122990.119,37.810631,-122.363443,,,,DPW,STREETS,No,N,NaN,NaN,NaN
1855,15311000,TREASURE ISLAND RD,TREASURE ISLAND RD: END to END (701 - 751),6023285.543,2122842.413,37.810220,-122.363779,,,,DPW,STREETS,No,N,NaN,NaN,NaN
1856,15312000,TREASURE ISLAND RD,TREASURE ISLAND RD: END to I-80 W ON RAMP \ MA...,6023377.543,2123201.128,37.811210,-122.363485,,,,DPW,STREETS,No,N,NaN,NaN,NaN
1857,15314000,NORTH GATE RD,NORTH GATE RD: I-80 W ON RAMP \ MACALLA RD \ S...,6022933.843,2123626.965,37.812355,-122.365050,,,,DPW,STREETS,No,N,NaN,NaN,NaN


We can't analyze these streets, so we'll remove them from the unaccepted streets dataset:

In [13]:
df_dpw = df_dpw.dropna(subset=['line'])

Below we're converting the unaccepted streets dataset to a GeoDataFrame, which allows us to work with the geometry column. 

In [14]:
df_dpw["geometry"] = df_dpw["line"].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df_dpw, geometry="geometry")

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_10698/3428783731.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dpw["geometry"] = df_dpw["line"].apply(wkt.loads)


In [15]:
gdf.head()

,cnn,street_name,location,x_coord,y_coord,latitude,longitude,bos_district,zip_code,sfar_neighborhood_desc,jurisdiction,layer,bike_path,accepted,line,active,supervisor_district,geometry
0,110000,01ST ST,01ST ST: CLEMENTINA ST to FOLSOM ST (245 - 299),6014174.520,2114753.853,37.787508,-122.394745,6,94105,South Beach,CALTRANS/DPW,STREETS,No,N,"LINESTRING (-122.395007842 37.787717615, -122....",True,6.0,"LINESTRING (-122.39501 37.78772, -122.39448 37..."
1,114000,01ST ST,01ST ST: HARRISON ST \ I-80 E ON RAMP to END (...,6014784.513,2114116.436,37.785792,-122.392589,6,94105,South Beach,CALTRANS/DPW,STREETS,No,N,"LINESTRING (-122.392936685 37.786067532, -122....",True,6.0,"LINESTRING (-122.39294 37.78607, -122.39224 37..."
2,140000,02ND ST,02ND ST: HARRISON ST to STILLMAN ST (400 - 459),6014173.689,2113442.524,37.783907,-122.394656,6,94107,South of Market,CALTRANS/DPW,STREETS,Yes,N,"LINESTRING (-122.395160622 37.784311013, -122....",True,6.0,"LINESTRING (-122.39516 37.78431, -122.39415 37..."
3,150000,03RD AVE,03RD AVE: END to LAKE ST (1 - 99),5994888.446,2114976.446,37.787028,-122.461486,1,94118,Lake,DPW,STREETS,No,N,"LINESTRING (-122.461501479 37.787239798, -122....",True,1.0,"LINESTRING (-122.46150 37.78724, -122.46147 37..."
4,159000,03RD AVE,03RD AVE: KEZAR DR \ LINCOLN WAY to LINCOLN WA...,5995166.956,2107405.925,37.766257,-122.459974,7,94117,Inner Sunset,DPW,STREETS,Yes,N,"LINESTRING (-122.459931488 37.76634884, -122.4...",True,7.0,"LINESTRING (-122.45993 37.76635, -122.46002 37..."


### calculting the length of each unaccepted street

Below I am assigning a CRS to the unaccepted streets dataset. The CRS is set to EPSG:4326, which is a common coordinate system. However, its measurements are in degrees and not in meters, so I need to convert it to a projected CRS that uses meters (32610). Once converted we can add a length column. 

In [16]:
gdf = gdf.set_crs("EPSG:4326")

gdf = gdf.to_crs("EPSG:32610")

gdf["length_m"] = gdf.geometry.length


In [17]:
print(gdf.crs)

EPSG:32610


Checking that the measuremnts look correct:

In [18]:
gdf.length_m.value_counts()

65.689769     1
49.668933     1
90.481978     1
69.028951     1
193.342833    1
             ..
52.593511     1
51.446495     1
211.393192    1
113.413481    1
48.561061     1
Name: length_m, Length: 1938, dtype: int64

In [19]:
gdf.length_m.sum()

220431.27980877284

In [20]:
gdf["length_m"].mean()

113.7416304482832

Below I am comparing two columns for the supervisor district, for some reason, the one from the streets data seems to capture more than the one from the DPW dataset, which includes blanks. The blanks in the streets one come from areas in water or just outside the city's / a district's jurisdiction. To capture the most street segments we'll use the one from the streets dataset. 

In [21]:
total_length_by_district = gdf.groupby('bos_district')['length_m'].sum().reset_index()

In [22]:
total_length_by_district_2 = gdf.groupby('supervisor_district')['length_m'].sum().reset_index()

In [23]:
total_length_by_district

,bos_district,length_m
0,,34079.470582
1,1,6795.323660
2,10,51250.132509
3,11,17849.209816
4,2,4765.697315
5,3,9336.740797
6,4,2856.955789
7,5,3396.525195
8,6,20377.139817
9,7,29424.070933


In [24]:
total_length_by_district_2

,supervisor_district,length_m
0,1.0,6795.323660
1,2.0,10848.119709
2,3.0,9381.182857
3,4.0,2417.856822
4,5.0,3396.525195
5,6.0,32842.243988
6,7.0,28528.247737
7,8.0,16073.784182
8,9.0,24389.908711
9,10.0,55077.870453


In [25]:
gdf.bos_district.value_counts()

10    461
7     252
9     244
      222
8     164
11    157
3     154
6     136
5      42
2      42
1      40
4      24
Name: bos_district, dtype: int64

In [26]:
gdf.supervisor_district.value_counts()

10.0    493
7.0     251
9.0     241
6.0     223
11.0    167
8.0     164
3.0     155
2.0      58
5.0      42
1.0      40
4.0      23
Name: supervisor_district, dtype: int64

In [27]:
blank_rows = gdf[gdf["bos_district"].apply(lambda x: isinstance(x, str) and x.strip() == '')]

blank_rows

,cnn,street_name,location,x_coord,y_coord,latitude,longitude,bos_district,zip_code,sfar_neighborhood_desc,jurisdiction,layer,bike_path,accepted,line,active,supervisor_district,geometry,length_m
40,19002000,14TH AVE,14TH AVE: WEDEMEYER ST to PARK BLVD (0 - 0),5991466.898,2115162.931,37.787342,-122.473337,,94129,Presidio,PRESIDIO/RECPARK/DPW,PARKS_NPS_PRESIDIO,Yes,N,"LINESTRING (-122.47335421 37.787491818, -122.4...",True,2.0,"LINESTRING (546371.367 4182367.808, 546374.507...",33.278919
59,1023000,20TH ST,20TH ST: LOUISIANA ST to UNNAMED 042 (300 - 399),6016741.428,2104935.968,37.760693,-122.385179,,,,DPW,STREETS,No,N,"LINESTRING (-122.38475499 37.760717793, -122.3...",True,10.0,"LINESTRING (554192.173 4179444.880, 554117.489...",74.931624
60,1024000,20TH ST,20TH ST: UNNAMED 042 to MICHIGAN ST (400 - 499),6016476.021,2104921.431,37.760638,-122.386096,,,,DPW,STREETS,No,N,"LINESTRING (-122.385603281 37.760667282, -122....",True,10.0,"LINESTRING (554117.489 4179438.785, 554030.721...",87.056265
143,2234000,ALVORD ST,ALVORD ST: END to CARROLL AVE (0 - 0),6019520.709,2088104.632,37.714630,-122.374401,,94124,NaN,DPW,PAPER_WATER,No,N,"LINESTRING (-122.374157371 37.714900715, -122....",True,NaN,"LINESTRING (555159.630 4174367.721, 555117.146...",73.748999
144,2235000,ALVORD ST,ALVORD ST: CARROLL AVE to DONNER AVE (0 - 0),6019364.737,2087895.556,37.714048,-122.374925,,94124,NaN,DPW,PAPER_WATER,No,N,"LINESTRING (-122.374643932 37.714359953, -122....",True,NaN,"LINESTRING (555117.146 4174307.438, 555068.014...",85.215571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1945,15329000,SUNNYDALE AVE,SUNNYDALE AVE: SANTOS ST to END (0 - 0),6006743.014,2087633.962,37.712627,-122.418532,,,,DPW,PAPER,No,N,"LINESTRING (-122.418038222 37.712487948, -122....",True,10.0,"LINESTRING (551293.750 4174075.092, 551206.459...",92.396144
1946,15331000,SUNRISE WAY,SUNRISE WAY: MALOSI ST to SANTOS ST (0 - 0),6006690.422,2086843.851,37.710454,-122.418658,,,,DPW,PAPER,No,N,"LINESTRING (-122.418243703 37.710337536, -122....",True,10.0,"LINESTRING (551277.121 4173836.395, 551203.908...",77.521950
1947,14348000,MACKY LN,MACKY LN: GOLDEN BELL WAY to END (0 - 0),6021870.888,2127869.609,37.823946,-122.369023,,,,DPW,STREETS,No,N,"LINESTRING (-122.368222763 37.822730801, -122....",True,6.0,"LINESTRING (555601.891 4186334.918, 555459.263...",304.158021
1948,14291000,MAYBECK ST,MAYBECK ST: AVENUE OF THE PALMS to GARDEN WALK...,6019867.686,2128150.234,37.824606,-122.375976,,,,DPW,STREETS,No,N,"LINESTRING (-122.376249932 37.824351379, -122....",True,6.0,"LINESTRING (554894.223 4186509.975, 554942.000...",74.272944


### calculating the area of each district

Below I'm reading in a supervisor district dataset to compare street segments with area. I'm running the same kind of reprojections to calculate the area of each district in square miles.

This file was obtained from [S.F. Open Data](https://data.sfgov.org/Geographic-Locations-and-Boundaries/Supervisor-Districts-2022-/f2zs-jevy/about_data)

In [28]:
gdf_supe = gpd.read_file('./data/supe22.geojson')

In [29]:
gdf_supe

,sup_dist_pad,sup_dist_num,sup_dist_name,data_loaded_at,sup_name,sup_dist,data_as_of,geometry
0,01,1.0,SUPERVISORIAL DISTRICT 1,2025-01-08 18:30:00,Connie Chan,1,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.48520 37.79020, -122.4848..."
1,02,2.0,SUPERVISORIAL DISTRICT 2,2025-01-08 18:30:00,Stephen Sherrill,2,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.47695 37.81098, -122.4766..."
2,03,3.0,SUPERVISORIAL DISTRICT 3,2025-01-08 18:30:00,Danny Sauter,3,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.41972 37.81113, -122.4195..."
3,04,4.0,SUPERVISORIAL DISTRICT 4,2025-01-08 18:30:00,Joel Engardio,4,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.51103 37.77056, -122.5110..."
4,05,5.0,SUPERVISORIAL DISTRICT 5,2025-01-08 18:30:00,Bilal Mahmood,5,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.42706 37.78879, -122.4254..."
5,06,6.0,SUPERVISORIAL DISTRICT 6,2025-01-08 18:30:00,Matt Dorsey,6,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.33154 37.78694, -122.3279..."
6,07,7.0,SUPERVISORIAL DISTRICT 7,2025-01-08 18:30:00,Myrna Melgar,7,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.46583 37.77261, -122.4654..."
7,08,8.0,SUPERVISORIAL DISTRICT 8,2025-01-08 18:30:00,Rafael Mandelman,8,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.42371 37.77270, -122.4236..."
8,09,9.0,SUPERVISORIAL DISTRICT 9,2025-01-08 18:30:00,Jackie Fielder,9,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.42225 37.77052, -122.4217..."
9,10,10.0,SUPERVISORIAL DISTRICT 10,2025-01-08 18:30:00,Shamann Walton,10,2025-01-08 18:30:00,"MULTIPOLYGON (((-122.38541 37.76725, -122.3858..."


In [30]:
print(gdf_supe.crs)

epsg:4326


In [31]:
gdf_supe = gdf_supe.set_crs("EPSG:4326", allow_override=True)

gdf_supe = gdf_supe.to_crs("EPSG:32610")

gdf_supe["area_sqm"] = gdf_supe.geometry.area
gdf_supe["area_sqmi"] = gdf_supe["area_sqm"] / 2.59e+6  


In [32]:
gdf_supe

,sup_dist_pad,sup_dist_num,sup_dist_name,data_loaded_at,sup_name,sup_dist,data_as_of,geometry,area_sqm,area_sqmi
0,01,1.0,SUPERVISORIAL DISTRICT 1,2025-01-08 18:30:00,Connie Chan,1,2025-01-08 18:30:00,"MULTIPOLYGON (((545326.945 4182662.139, 545358...",1.008838e+07,3.895126
1,02,2.0,SUPERVISORIAL DISTRICT 2,2025-01-08 18:30:00,Stephen Sherrill,2,2025-01-08 18:30:00,"MULTIPOLYGON (((546039.995 4184972.363, 546069...",1.315037e+07,5.077364
2,03,3.0,SUPERVISORIAL DISTRICT 3,2025-01-08 18:30:00,Danny Sauter,3,2025-01-08 18:30:00,"MULTIPOLYGON (((551077.283 4185018.963, 551090...",5.375721e+06,2.075568
3,04,4.0,SUPERVISORIAL DISTRICT 4,2025-01-08 18:30:00,Joel Engardio,4,2025-01-08 18:30:00,"MULTIPOLYGON (((543063.801 4180470.892, 543065...",2.605506e+08,100.598706
4,05,5.0,SUPERVISORIAL DISTRICT 5,2025-01-08 18:30:00,Bilal Mahmood,5,2025-01-08 18:30:00,"MULTIPOLYGON (((550446.742 4182535.475, 550590...",4.602155e+06,1.776894
5,06,6.0,SUPERVISORIAL DISTRICT 6,2025-01-08 18:30:00,Matt Dorsey,6,2025-01-08 18:30:00,"MULTIPOLYGON (((558859.079 4182386.607, 559182...",9.189229e+06,3.547965
6,07,7.0,SUPERVISORIAL DISTRICT 7,2025-01-08 18:30:00,Myrna Melgar,7,2025-01-08 18:30:00,"MULTIPOLYGON (((547043.753 4180720.333, 547075...",2.093441e+07,8.082782
7,08,8.0,SUPERVISORIAL DISTRICT 8,2025-01-08 18:30:00,Rafael Mandelman,8,2025-01-08 18:30:00,"MULTIPOLYGON (((550752.354 4180752.555, 550755...",9.351741e+06,3.610711
8,09,9.0,SUPERVISORIAL DISTRICT 9,2025-01-08 18:30:00,Jackie Fielder,9,2025-01-08 18:30:00,"MULTIPOLYGON (((550883.135 4180511.261, 550928...",8.558112e+06,3.304290
9,10,10.0,SUPERVISORIAL DISTRICT 10,2025-01-08 18:30:00,Shamann Walton,10,2025-01-08 18:30:00,"MULTIPOLYGON (((554129.384 4180169.082, 554093...",1.943980e+07,7.505714


In [33]:
gdf_supe = gdf_supe[['sup_dist_num', 'area_sqmi', 'area_sqm']]

In [34]:
gdf_supe.dtypes

sup_dist_num     object
area_sqmi       float64
area_sqm        float64
dtype: object

In [35]:
gdf_supe['sup_dist_num'] = gdf_supe['sup_dist_num'].astype(float)

/var/folders/_0/x7xh_2c16jq4chdcqrg28fr40000gn/T/ipykernel_10698/4159580652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_supe['sup_dist_num'] = gdf_supe['sup_dist_num'].astype(float)


### stats

This is the total length of all the unaccepted streets in each district: 

In [36]:
total_length_by_district_2.sort_values(by='length_m', ascending=False, inplace=True)

total_length_by_district_2

,supervisor_district,length_m
9,10.0,55077.870453
5,6.0,32842.243988
6,7.0,28528.247737
8,9.0,24389.908711
10,11.0,20021.764355
7,8.0,16073.784182
1,2.0,10848.119709
2,3.0,9381.182857
0,1.0,6795.323660
4,5.0,3396.525195


This shows the same thing with the percentage of unaccepted streets in each district, compared to the total length of all streets in the city: 

In [37]:
total_length_by_district_2['percentage'] = (total_length_by_district_2['length_m'] / total_length_by_district_2['length_m'].sum()) * 100

total_length_by_district_2

,supervisor_district,length_m,percentage
9,10.0,55077.870453,26.255960
5,6.0,32842.243988,15.656100
6,7.0,28528.247737,13.599592
8,9.0,24389.908711,11.626820
10,11.0,20021.764355,9.544498
7,8.0,16073.784182,7.662472
1,2.0,10848.119709,5.171366
2,3.0,9381.182857,4.472068
0,1.0,6795.323660,3.239373
4,5.0,3396.525195,1.619144


The total length of unaccepted streets in the city, in meters:

In [38]:
total_length_by_district_2.length_m.sum()

209772.82767035303

The same thing, in miles:

In [39]:
total_length_by_district_2.length_m.sum() / 1609.34  # 1 mile = 1609.34 meters

130.34711600429557

In [40]:
total_length_by_district_2['length_mi'] = total_length_by_district_2['length_m'] / 1609.34  # Convert meters to miles

Here I'm adding the area data to the district totals: 

In [41]:
merged_gdf = total_length_by_district_2.merge(gdf_supe, left_on='supervisor_district', right_on='sup_dist_num', how='left')

In [42]:
merged_gdf

,supervisor_district,length_m,percentage,length_mi,sup_dist_num,area_sqmi,area_sqm
0,10.0,55077.870453,26.255960,34.223887,10.0,7.505714,1.943980e+07
1,6.0,32842.243988,15.656100,20.407275,6.0,3.547965,9.189229e+06
2,7.0,28528.247737,13.599592,17.726675,7.0,8.082782,2.093441e+07
3,9.0,24389.908711,11.626820,15.155224,9.0,3.304290,8.558112e+06
4,11.0,20021.764355,9.544498,12.440979,11.0,3.396866,8.797882e+06
5,8.0,16073.784182,7.662472,9.987811,8.0,3.610711,9.351741e+06
6,2.0,10848.119709,5.171366,6.740726,2.0,5.077364,1.315037e+07
7,3.0,9381.182857,4.472068,5.829211,3.0,2.075568,5.375721e+06
8,1.0,6795.323660,3.239373,4.222429,1.0,3.895126,1.008838e+07
9,5.0,3396.525195,1.619144,2.110508,5.0,1.776894,4.602155e+06


For every square mile, this is how many miles of unaccepted streets there are in each district:

In [43]:
merged_gdf['length_per_sqmi'] = merged_gdf['length_mi'] / merged_gdf['area_sqmi']

In [44]:
merged_gdf.sort_values(by='length_per_sqmi', ascending=False, inplace=True)

merged_gdf

,supervisor_district,length_m,percentage,length_mi,sup_dist_num,area_sqmi,area_sqm,length_per_sqmi
1,6.0,32842.243988,15.656100,20.407275,6.0,3.547965,9.189229e+06,5.751826
3,9.0,24389.908711,11.626820,15.155224,9.0,3.304290,8.558112e+06,4.586530
0,10.0,55077.870453,26.255960,34.223887,10.0,7.505714,1.943980e+07,4.559711
4,11.0,20021.764355,9.544498,12.440979,11.0,3.396866,8.797882e+06,3.662487
7,3.0,9381.182857,4.472068,5.829211,3.0,2.075568,5.375721e+06,2.808490
5,8.0,16073.784182,7.662472,9.987811,8.0,3.610711,9.351741e+06,2.766162
2,7.0,28528.247737,13.599592,17.726675,7.0,8.082782,2.093441e+07,2.193140
6,2.0,10848.119709,5.171366,6.740726,2.0,5.077364,1.315037e+07,1.327603
9,5.0,3396.525195,1.619144,2.110508,5.0,1.776894,4.602155e+06,1.187751
8,1.0,6795.323660,3.239373,4.222429,1.0,3.895126,1.008838e+07,1.084029


Dropping a column that is not needed anymore:

In [45]:
merged_gdf = merged_gdf.drop(columns=['sup_dist_num'])

In [46]:
merged_gdf

,supervisor_district,length_m,percentage,length_mi,area_sqmi,area_sqm,length_per_sqmi
1,6.0,32842.243988,15.656100,20.407275,3.547965,9.189229e+06,5.751826
3,9.0,24389.908711,11.626820,15.155224,3.304290,8.558112e+06,4.586530
0,10.0,55077.870453,26.255960,34.223887,7.505714,1.943980e+07,4.559711
4,11.0,20021.764355,9.544498,12.440979,3.396866,8.797882e+06,3.662487
7,3.0,9381.182857,4.472068,5.829211,2.075568,5.375721e+06,2.808490
5,8.0,16073.784182,7.662472,9.987811,3.610711,9.351741e+06,2.766162
2,7.0,28528.247737,13.599592,17.726675,8.082782,2.093441e+07,2.193140
6,2.0,10848.119709,5.171366,6.740726,5.077364,1.315037e+07,1.327603
9,5.0,3396.525195,1.619144,2.110508,1.776894,4.602155e+06,1.187751
8,1.0,6795.323660,3.239373,4.222429,3.895126,1.008838e+07,1.084029


### total streets

Going back to our streets dataset with **all** streets in San Francisco, so that we can compare unaccepted streets with all streets.

Making the same adjustments to this dataset as the other ones so that we can obtain correct measurements in meters, and covert to miles. I am also filtering the streets data to only include "active" streets (the dataset includes retired streets as well)

In [47]:
gdf_streets = gpd.GeoDataFrame(df_streets, geometry=gpd.GeoSeries.from_wkt(df_streets['line']))

gdf_streets = gdf_streets[gdf_streets['active'] == True]

gdf_streets = gdf_streets.set_crs("EPSG:4326", allow_override=True)

gdf_streets = gdf_streets.to_crs("EPSG:32610")

gdf_streets["length_m"] = gdf_streets.geometry.length

gdf_streets["length_mi"] = gdf_streets["length_m"] / 1609.34  # Convert meters to miles

In [48]:
gdf_streets.active.value_counts()

True    16367
Name: active, dtype: int64

In [49]:
total_length_all_streets_by_district = gdf_streets.groupby('supervisor_district')['length_mi'].sum().reset_index()

This is the total length of all streets in each district:

In [50]:
total_length_all_streets_by_district

,supervisor_district,length_mi
0,1.0,85.917765
1,2.0,121.167011
2,3.0,71.081974
3,4.0,118.869783
4,5.0,54.044416
5,6.0,114.159281
6,7.0,178.138472
7,8.0,111.248616
8,9.0,110.720127
9,10.0,163.929841


This will join the summary table to our big summary table:

In [51]:
merged_gdf = merged_gdf.merge(total_length_all_streets_by_district, on='supervisor_district', how='left', suffixes=('', '_streets'))

This calcualtes the percentage of unaccepted streets compared to all streets in each district:

In [52]:
merged_gdf['percentage_length'] = (merged_gdf['length_mi'] / merged_gdf['length_mi_streets']) * 100

In [53]:
merged_gdf

,supervisor_district,length_m,percentage,length_mi,area_sqmi,area_sqm,length_per_sqmi,length_mi_streets,percentage_length
0,6.0,32842.243988,15.656100,20.407275,3.547965,9.189229e+06,5.751826,114.159281,17.876142
1,9.0,24389.908711,11.626820,15.155224,3.304290,8.558112e+06,4.586530,110.720127,13.687868
2,10.0,55077.870453,26.255960,34.223887,7.505714,1.943980e+07,4.559711,163.929841,20.877155
3,11.0,20021.764355,9.544498,12.440979,3.396866,8.797882e+06,3.662487,112.281881,11.080130
4,3.0,9381.182857,4.472068,5.829211,2.075568,5.375721e+06,2.808490,71.081974,8.200688
5,8.0,16073.784182,7.662472,9.987811,3.610711,9.351741e+06,2.766162,111.248616,8.977920
6,7.0,28528.247737,13.599592,17.726675,8.082782,2.093441e+07,2.193140,178.138472,9.951065
7,2.0,10848.119709,5.171366,6.740726,5.077364,1.315037e+07,1.327603,121.167011,5.563169
8,5.0,3396.525195,1.619144,2.110508,1.776894,4.602155e+06,1.187751,54.044416,3.905136
9,1.0,6795.323660,3.239373,4.222429,3.895126,1.008838e+07,1.084029,85.917765,4.914500


In [54]:
length_overview = merged_gdf[['supervisor_district', 'length_mi', 'length_mi_streets', 'percentage_length']]

In [55]:
length_overview = length_overview.round(1)
length_overview

,supervisor_district,length_mi,length_mi_streets,percentage_length
0,6.0,20.4,114.2,17.9
1,9.0,15.2,110.7,13.7
2,10.0,34.2,163.9,20.9
3,11.0,12.4,112.3,11.1
4,3.0,5.8,71.1,8.2
5,8.0,10.0,111.2,9.0
6,7.0,17.7,178.1,10.0
7,2.0,6.7,121.2,5.6
8,5.0,2.1,54.0,3.9
9,1.0,4.2,85.9,4.9


### exports

Exporting a more readable version of the summary table to a CSV file:

In [56]:
length_overview.to_csv('./output/length_overview.csv', index=False)

Exporting the rows that don't fall into any district to a CSV file (to look at in QGIS): 

In [57]:
blank_rows.to_file('./output/no_district.geojson', driver='GeoJSON')

Exporting our main geoDataFrame with all the unaccepted streets to a geojson file, for mapping: 

In [58]:
gdf.to_file('./output.output.geojson', driver='GeoJSON')